In [1]:
import json
import jsonlines
import sys

# dirs = ["..", "/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/my_inferencing/create_prompt_llm"]
# for _dir in dirs:
#     if _dir not in sys.path:
#         sys.path.append(_dir)
import prompt_rag

search_engine = "brave"

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)


In [2]:
# prompt_rag.update_train_search_llm( 
#     "SOLAR-10.7B-Instruct-v1.0", "solar", port=8001, 
#     data_search=data_search, data_search_llm=data_search_llm) 



合并多个文件的先验知识

In [3]:
# bing search llm
# v2: k = 5
# v3: 不对时间排序

# brave search llm
# v1: k = 5
# v2: k = 5 随机选取

sort = False
prior_knowledge_list = []

K = 5
prior_knowledge_version = 1

num_part = 3

# for i in range(1, num_part + 1):
#     with open(f"{search_engine}/prior_knowledge/random/train_search_llm_v{i}.json", "r") as f:
#         prior_knowledge_list.extend(json.load(f))

# for i, item in enumerate(data_search_llm):
#     if item["claim"] != prior_knowledge_list[i]["claim"].strip():
#         print(i)
#         print(item["claim"])
#         print(prior_knowledge_list[i]["claim"])
#         raise Exception()
#     else:
#         item[f"prior_knowledge_solar_v{prior_knowledge_version}_K={K}"] = prior_knowledge_list[i]["prior_knowledge_solar"]

# data_search_llm[0]


In [4]:
# prompt_rag.save_search_llm(data_search_llm, search_engine)


创建数据（带有先验知识的Prompt）以微调LLM

In [5]:
# dict_list = []

# for i, item in enumerate(data_search_llm):
    
#     if int(item["label"]) != 1:
#         prompt = prompt_rag.get_prompt_with_prior_knowledge(
#             item["claim"], 
#             search_engine,
#             data_search[i][f"{search_engine}_search_results"], 
#             item[f"prior_knowledge_solar_v{prior_knowledge_version}_K={K}"], 
#             K=K,
#             claim_date=item["date"],
#             sort=sort,
#             ids=data_search[i]["random_ids"] # 随机选取
#         )
#         label = "TRUE." if int(item["label"]) == 2 else "FALSE."
#         dict_list.append({"query": prompt, "response": label})
# print(dict_list[0]["query"])
# len(dict_list), dict_list[:2]


In [6]:
# LLM_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/"
# data_version = "2"
# with jsonlines.open(
#     LLM_dir + \
#     f"swift/examples/pytorch/llm/my_data/with_solar_info/{search_engine}/data{data_version}.jsonl", mode="w") as file_jsonl:
#     for line in dict_list:
#         file_jsonl.write(line)


测试模型的先验知识生成效果：一次提问

In [7]:
import torch
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

model_type = CustomModelType.solar_instruct_10_7b
llm_engine = get_vllm_engine(
    model_type, 
    torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=4096,
    temperature=0,
)


2024-03-27 20:36:41,320 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-03-27 20:36:41,321 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-03-27 20:36:41,357 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/SOLAR-10.7B-Instruct-v1.0
[INFO:swift] Setting torch_dtype: torch.float16
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/SOLAR-10.7B-Instruct-v1.0",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 48,
  "num_key_value

INFO 03-27 20:36:41 config.py:433] Custom all-reduce kernels are temporarily disabled due to stability issues. We will re-enable them once the issues are resolved.


2024-03-27 20:36:44,810	INFO worker.py:1724 -- Started a local Ray instance.


INFO 03-27 20:36:46 llm_engine.py:87] Initializing an LLM engine with config: model='/home/css/models/SOLAR-10.7B-Instruct-v1.0', tokenizer='/home/css/models/SOLAR-10.7B-Instruct-v1.0', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=42)


(raylet) [2024-03-27 20:36:53,707 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85305532416; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-03-27 20:37:03,754 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85304872960; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-03-27 20:37:13,814 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85304832000; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-03-27 20:37:23,907 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85304795136; capacity: 19

INFO 03-27 20:37:41 llm_engine.py:357] # GPU blocks: 7315, # CPU blocks: 2730


(raylet) [2024-03-27 20:37:43,979 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85303365632; capacity: 1967317549056. Object creation will fail if spilling is required.


INFO 03-27 20:37:44 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-27 20:37:44 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(RayWorkerVllm pid=1737485) INFO 03-27 20:37:44 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(RayWorkerVllm pid=1737485) INFO 03-27 20:37:44 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforci

(raylet) [2024-03-27 20:37:53,988 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85302706176; capacity: 1967317549056. Object creation will fail if spilling is required.


In [8]:
K = 5
def get_id(claim):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            return i

i = 0
# i = get_id("The Government of Mexico City offers $1,000 weekly if you stay at home.")

claim = data_search[i]["claim"]
search_results = data_search[i][f"{search_engine}_search_results"]

prompt = prompt_rag.get_prompt_for_generating_prior_knowledge(
    claim, data_search[i]["date"], search_engine, search_results, 
    K=K, sort=False, 
    # ids=data_search[i]["random_ids"],
    ids=None
)

request_list = [{'query': prompt}, ]

print(prompt)

resp_list = inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=False,
    # verbose=True, prompt_prefix="", output_prefix=""
)
print(resp_list[0]["response"].strip())
# res = al.get_response(prompt, "SOLAR-10.7B-Instruct-v1.0", cache_seed=None, port=8001)
# print(res.strip())


Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.

CLAIM:
Publication date: 2020-08-13
Content: Illinois Strengthens Face Mask Rules in Businesses

INFORMATION:
Information 1:
Publication date: 2013-08-06
Title: Illinois Strengthens Face Mask Rules in Businesses - WebMD
Content:
COVID-19 is a new type of coronavirus that causes mild to severe case

(raylet) [2024-03-27 20:38:03,997 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85302292480; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-03-27 20:38:14,005 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85302026240; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-03-27 20:38:24,013 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85301501952; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-03-27 20:38:34,021 E 1733951 1733990] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-27_20-36-41_918744_1733345 is over 95% full, available space: 85301190656; capacity: 19

CLAIM: Publication date: 2020-08-13, Content: Illinois Strengthens Face Mask Rules in Businesses

To accurately classify this claim, we need to analyze the relevant information provided. 

Information 1: This is from 2013 and is not directly related to the claim about Illinois strengthening face mask rules in businesses in 2020. It provides general information about COVID-19 symptoms, risk factors, and prevention.

Information 2: This information confirms that Illinois lifted its indoor mask requirement on February 28, 2022, except where required by specific laws, rules, and regulations. It also mentions that the state was optimistic about this decision due to the decreasing trends of COVID-19 cases. The information does not directly contradict the claim about strengthening face mask rules in 2020, but it provides context about the situation in 2021-2022.

Information 3: This information reveals that on September 3, 2021, the Governor of Illinois signed an Executive Order requiring fac